In [1]:
#fix weeks on list to what we are finding, how long weeks on list
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [13]:
file_path = Path('cleaned_best_seller_code.csv')
df = pd.read_csv(file_path)

In [15]:
#binning the data
bins= [0,4,15,25,55,610]
labels= ["0-4 weeks","5-15 weeks","16-25 weeks","26-55 weeks","56-610 weeks"]
df['bin_weeks_on_list']= pd.cut(df['weeks_on_list'] , bins=bins, labels= labels, include_lowest=True)

In [16]:
#getting X and y
X = df.drop('bin_weeks_on_list',axis= 1)
X = pd.get_dummies(X)
y= df['bin_weeks_on_list']

In [19]:
X.describe()

,rank,price,weeks_on_list,published_date_2010-01-03,published_date_2010-01-10,published_date_2010-01-17,published_date_2010-01-24,published_date_2010-01-31,published_date_2010-02-07,published_date_2010-02-14,...,description_ Is this the end of humanity? It sure looks that way when what is left of mankind must go into hiding to escape the giant humanoids known as the Titans.,"description_ Searching for her sister’s killer, a former F.B.I. researcher finds a link between scores of apparently unconnected unsolved cases. Now she must convince her boss and her former boyfriend that this killer is real.",description_ The final book of the Hollows series.,"description_’Night, Baby Llama.","description_“Barefoot Contessa” recipes with instructions on preparing, assembling or cooking meals ahead of time.",description_“Dear Sugar” advice columns.,description_“Hawkguy” rallies his neighbors and his brother to help protect his building from hired assassins and tracksuit–wearing thugs.,"description_“He’ll ask for some apple juice to go with it,” for starters.","description_“He’ll ask for some apple juice to go with it,” to start things off.","description_“Holy licensing, Batman!” All of the iconic hallmarks and camp sensibilities of the 1960s “Batman” television program are transported to the printed page in this series."
count,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,...,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000,51997.000000
mean,2.999731,4.533286,27.229859,0.001327,0.001346,0.001346,0.001346,0.001346,0.001346,0.001346,...,0.000019,0.000019,0.000019,0.000019,0.000327,0.000019,0.000019,0.000135,0.000154,0.000058
std,1.415247,9.328902,67.540999,0.036404,0.036667,0.036667,0.036667,0.036667,0.036667,0.036667,...,0.004385,0.004385,0.004385,0.004385,0.018079,0.004385,0.004385,0.011602,0.012403,0.007596
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,150.000000,607.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
#check the stats on week on list
y.describe()

count         51997
unique            5
top       0-4 weeks
freq          29565
Name: bin_weeks_on_list, dtype: object

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [22]:
#fit the model for BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc = brfc.fit(X_train, y_train)


In [23]:
#predict
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9703041421691181

In [24]:
confusion_matrix(y_test, y_pred)

array([[7313,    5,    3,   56,    0],
       [   0,  722,   12,    7,    0],
       [   0,   43, 1016,    0,   10],
       [   0,   53,    3, 2041,    0],
       [   0,    0,   65,    0, 1651]], dtype=int64)

In [25]:
print(classification_report_imbalanced(y_test,y_pred))

                    pre       rec       spe        f1       geo       iba       sup

   0-4 weeks       1.00      0.99      1.00      1.00      1.00      0.99      7377
 16-25 weeks       0.88      0.97      0.99      0.92      0.98      0.96       741
 26-55 weeks       0.92      0.95      0.99      0.94      0.97      0.94      1069
  5-15 weeks       0.97      0.97      0.99      0.97      0.98      0.97      2097
56-610 weeks       0.99      0.96      1.00      0.98      0.98      0.96      1716

 avg / total       0.98      0.98      1.00      0.98      0.99      0.98     13000

